In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
from lxml import html
import scrapy
from time import sleep
import urllib3
import json
from selenium import webdriver
import random

In [2]:
def parser(link):

    encabezados = {
    'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/44.0.2403.157 Safari/537.36'
    }

    resp = requests.get(link, headers=encabezados, verify=False)
    resp = resp.text

    #soup = get_Soup('https://minciencias.gov.co/convocatorias/todas')
    parser = html.fromstring(resp)
    urllib3.disable_warnings()

    return parser

In [3]:
pag = 'http://www.fapepi.pi.gov.br/editais/page/1/'

parser_pag_principal = parser(pag)

pag_final = parser_pag_principal.xpath('//a[@class="page-numbers"]/@href')[-1]

pag_final =pag_final.split('page/')[1].split('/')[0]

In [4]:
n = 1
links_proyectos = []
titulos_proyectos = []
fechas_cierre_proyectos = []
fechas_apertura_proyectos = []

while n <= int(pag_final):
    pag = 'http://www.fapepi.pi.gov.br/editais/page/' + str(n) + '/'

    parser_pag = parser(pag)

    # Links     
    links = parser_pag.xpath('//a[contains(text(), "Continuar lendo")]/@href')

    #titulos
    titulos = parser_pag.xpath('//h2//a/@title')

    #Fechas
    fecha_apertura = parser_pag.xpath('//ul//li[@class="meta-date"]/text()')

    fecha_cierre_brutas = parser_pag.xpath('//ul//li[@class="meta-mod-date"]//text()')

    fecha_cierre= []
    for f in fecha_cierre_brutas:
        if f != 'Post last modified:':
            fecha_cierre.append(f)

    links_proyectos += links
    titulos_proyectos += titulos
    fechas_cierre_proyectos += fecha_cierre
    fechas_apertura_proyectos += fecha_apertura

    n+=1

In [5]:
descripcion_proyectos = []
pdfs_proyectos = []

for link in links_proyectos:
    
    parser_proyecto = parser(link)

    #Pdfs
    pdfs_brutos = parser_proyecto.xpath('//a//@href')

    pdfs= ''
    for f in pdfs_brutos:
        if f.endswith('.pdf'):
            pdfs = pdfs + f + ', '
    pdfs = pdfs.strip(', ')

    descripcion_bruta = parser_proyecto.xpath('//div[@id="content-wrap"]//p/text()')

    descripcion = ''
    for d in descripcion_bruta:
        d = d.strip()
        if d != '':
            descripcion = descripcion + d + ' '

    descripcion = descripcion.strip(' ')

    descripcion_proyectos.append(descripcion)
    pdfs_proyectos.append(pdfs)


In [7]:
fapepi = pd.DataFrame()

fapepi['Link'] = links_proyectos 
fapepi['Título'] = titulos_proyectos
fapepi['Fecha Cierre'] = fechas_cierre_proyectos
fapepi['Fecha Apertura'] = fechas_apertura_proyectos
fapepi['Descripción'] = descripcion_proyectos
fapepi['PDFs'] = pdfs_proyectos
fapepi.to_excel('brasil_fapepi.xlsx')